In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os
import cv2
import sys
import matplotlib.image as mpimg
from PIL import Image
import scipy
from scipy import ndimage
import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse, FileResponse

import pickle
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from configs import *
warnings.filterwarnings('ignore')
%matplotlib inline


In [2]:
# Load Logstic Model we have trained
filename = "SGD Ridge Regression L2 Norm classifier_img_size_300_300_3{'alpha': 1.0, 'max_iter': 2500}_validation f1_scor_0.7746478873239437_.sav"
SGD_l2_ridge_trained_with_img_size_300_300_3 = pickle.load(open("saved_model_weights/" + filename, 'rb'))

# filename = 'Shallow_NN_tanh_and_sigmoid.h5'
# Shallow_NN_tanh_1_hidden_layers = keras.models.load_model( "saved_model_weights/" + filename)

# filename = 'Shallow_NN_relu_and_sigmoid.h5'
# Shallow_NN_2_relu_1_hidden_layers = keras.models.load_model( "saved_model_weights/" + filename)

# filename = 'Shallow_NN_sigmoid_512_256_1__neurons.h5'
# Shallow_NN_3_sigmoid_2_hidden_layers = keras.models.load_model( "saved_model_weights/" + filename)

In [3]:
SAVE_DIR = "images_uploaded/"
resized_images = "resized_images/"
write_on_images = "write_on_images/"

resized_dim = 300 
image_features  = resized_dim * resized_dim * 3 

In [4]:

app = FastAPI(title='Deploying a ML Model with FastAPI')


class Model(str, Enum):
    sgd_Logistic_Regression = "SGD Ridge Regression"
#     Shallow_NN_tanh_1_hidden_layers      = "Shallow_NN_tanh_1_hidden_layers"
#     Shallow_NN_2_relu_1_hidden_layers    = "Shallow_NN_2_relu_1_hidden_layers"
#     Shallow_NN_3_sigmoid_2_hidden_layers = "Shallow_NN_3_sigmoid_2_hidden_layers"
#     Deep_NN             = "Deep_NN"



@app.get("/")
def home():
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:5007/docs."


@app.post("/predict_me") 
async def prediction(model: Model, file: UploadFile = File(...)):
    
    # Handle non images uploaded
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png", "JPG", "JPEG", "PNG")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Unsupported file provided.")
    # save uploaded image
    contents = await file.read() 
    # example of how you can save the file
    with open(f"{SAVE_DIR}{file.filename}", "wb") as f:
        f.write(contents)
    print("=====================")
    # read saved image using another library for resize into fixed size and save it again
    img = Image.open(f"{SAVE_DIR}{file.filename}")
    img = img.resize((resized_dim, resized_dim), Image.ANTIALIAS)
    img.save(SAVE_DIR + resized_images  +   file.filename)
    
    # Open resized fixed image, reshape it into vector and scale pixel intensity
    predict_image = mpimg.imread(f"{SAVE_DIR}{resized_images}{file.filename}")
    image_0_1_vector = predict_image.reshape(-1, 1)/255
    
    print("="*50)
    print(model)
    # predict using our Logistic Regression model
    if model == "SGD Ridge Regression":
        my_predicted_image = SGD_l2_ridge_trained_with_img_size_300_300_3.predict(image_0_1_vector.T)
#     elif model == "Shallow_NN_tanh_1_hidden_layers":
#         my_predicted_image = np.argmax(Shallow_NN_tanh_and_sigmoid.predict(image_0_1_vector.T), axis=1)
#     elif model == "Shallow_NN_2_relu_1_hidden_layers":
#         my_predicted_image = np.argmax(Shallow_NN_relu_and_sigmoid.predict(image_0_1_vector.T), axis=1)
#     elif model == "Shallow_NN_3_sigmoid_2_hidden_layers":
#         my_predicted_image = np.argmax(Shallow_NN_3_sigmoid_2_hidden_layers.predict(image_0_1_vector.T), axis=1)
        
    print(my_predicted_image)
    my_predicted_image = ["Me WOW :)!!!!!!!!" if my_predicted_image==1 else "Not Me:("][0]
    print(my_predicted_image)
    file_image = cv2.imread(f'images_uploaded/{filename}', cv2.IMREAD_UNCHANGED)
    # Write on image if its me or its not me
    cv2.putText(file_image, my_predicted_image, (10,200), cv2.FONT_HERSHEY_COMPLEX_SMALL, 6,  (170,125, 145, 120), 5)
    
    # save image after write on it
    cv2.imwrite(f'images_uploaded/{write_on_images}{filename}', file_image)
    
    # Return the image to be displayed in response
    return FileResponse(f'images_uploaded/{write_on_images}{filename}')

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!    
uvicorn.run(app, host=host, port=5004)

INFO:     Started server process [29314]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5004 (Press CTRL+C to quit)


INFO:     127.0.0.1:55876 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:55876 - "GET /openapi.json HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55878 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55884 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55886 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55888 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55890 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55892 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Mode

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/abdelrahman/.local/lib/python3.6/site-packages/uvicorn/protocols/http/h11_impl.py", line 373, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/home/abdelrahman/.local/lib/python3.6/site-packages/uvicorn/middleware/proxy_headers.py", line 75, in __call__
    return await self.app(scope, receive, send)
  File "/home/abdelrahman/.local/lib/python3.6/site-packages/fastapi/applications.py", line 208, in __call__
    await super().__call__(scope, receive, send)
  File "/home/abdelrahman/.local/lib/python3.6/site-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/abdelrahman/.local/lib/python3.6/site-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc
  File "/home/abdelrahman/.local/lib/python3.6/site-packages/starlette/middleware/errors.py", line 159, in __call__
    awai

Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55916 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[0]
Not Me:(
INFO:     127.0.0.1:55918 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
Model.sgd_Logistic_Regression
[1]
Me WOW :)!!!!!!!!
INFO:     127.0.0.1:55920 - "POST /predict_me?model=SGD%20Ridge%20Regression HTTP/1.1" 200 OK
